<a href="https://colab.research.google.com/github/daniloabramo/agente-ia-para-comunicacao/blob/main/comunicacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# -*- coding: utf-8 -*-
"""Cópia de Imersão IA Alura + Google Gemini - Aula 05 - Agentes (Final).ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1UeKPKYTjcE86hEmfW1NDFgRgwZNnJkQD
"""

# Commented out IPython magic to ensure Python compatibility.
# %pip -q install google-genai

# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, HTML # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import json # Adicionado para processar JSON na função gerar_html_resultado

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def agente_pesquisador_linguagem (idade, estado):
    pesquisador_linguagem = Agent(
        name="pesquisador_linguagem",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é organizar um roteiro de pesquisa para
        encontrar os sites com as informações recentes, sobre os termos, gírias e significados
        da mesma utilizada no estado e na idade aproximada.

        Use a ferramenta de busca do google (google_search) para encontrar informações sobre:
        1. Gírias e expressões populares usadas por pessoas da idade informada no estado informado
        2. Padrões de comunicação e linguagem típicos dessa faixa etária e região
        3. Exemplos de como essa linguagem é empregada em contextos informais e formais

        Retorne uma lista organizada com os termos, gírias e expressões mais relevantes,
        bem como seus significados e contextos de uso.
        """,
        description="Agente que pesquisa padrões linguísticos regionais e etários",
        tools=[google_search]
    )

    entrada = f"Faça uma pesquisa sobre termos, gírias e padrões de linguagem utilizados por pessoas com aproximadamente {idade} anos no estado de {estado}."

    resultado = call_agent(pesquisador_linguagem, entrada)
    return resultado

# --- AGENTE 2: PESQUISADOR DE ÁREA E OBJETIVO ---
def agente_pesquisador_area_obje(area, objetivo):
    pesquisador_area = Agent(
        name="pesquisador_area_objetivo",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é organizar um roteiro de pesquisa para
        encontrar os sites com as informações recentes, adequadas para a área informada e para
        o objetivo informado.

        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google
        (google_search)para recuperar as últimas publicações informativas muito relevantes sobre
        o objetivo dentro da área descrita abaixo.

        Foque em no máximo 4 publicações informativas, com base na relevância das métricas do
        google, trazendo conteúdos bem rankeados pelo algoritmo como conteúdo de qualidade,
        procure conteúdos escritos de acordo com o roteiro informado.
        """,
        description="Agente que pesquisa pela área e objetivo informado no Google",
        tools=[google_search]
    )

    entrada = f"Área: {area}\nObjetivo: {objetivo}\nFaça uma pesquisa sobre as informações mais recentes e relevantes para este objetivo nesta área."

    resultado = call_agent(pesquisador_area, entrada)
    return resultado

# --- AGENTE 3: PESQUISADOR DE TERMOS DE USO COMUM ---
def agente_pesquisador_termos_comuns(idade, estado, area):
    pesquisador_termos = Agent(
        name="pesquisador_termos_comuns",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google
        (google_search)para recuperar as últimas publicações e postagens em fóruns e redes
        sociais para extrair as informações do roteiro de pesquisa.

        Foque em no máximo 2 publicações, com base na relevância das métricas do google,
        trazendo conteúdos bem rankeados pelo algoritmo como conteúdo de qualidade, procure
        conteúdos escritos de acordo com o roteiro informado.

        Busque por termos, jargões e expressões técnicas comumente utilizadas na área indicada,
        especialmente os que sejam populares entre pessoas da idade e estado informados.
        """,
        description="Agente que pesquisa termos comuns utilizados pelo usuário",
        tools=[google_search]
    )

    entrada = f"Pesquise publicações e postagens em fóruns e redes sociais para identificar termos comuns utilizados por pessoas com aproximadamente {idade} anos, no estado de {estado}, que atuam ou se interessam pela área de {area}."

    resultado = call_agent(pesquisador_termos, entrada)
    return resultado

# --- AGENTE 4: ANALISADOR E OTIMIZADOR DE TEXTO ---
def agente_analisador_texto(texto_original, idade, estado, area, objetivo, info_linguagem, info_area, info_termos):
    analisador = Agent(
        name="analisador_texto",
        model="gemini-2.0-flash",  # Usando modelo disponível no tier gratuito
        instruction="""
        Você é um especialista em análise e adaptação de textos. Com base no texto fornecido
        e nas informações sobre área, objetivo, idade e estado do usuário, você deve gerar
        uma versão melhorada e mais adequada para o objetivo pretendido pelo usuário.

        Seu trabalho é:

        1. Analisar o texto original
        2. Identificar problemas gramaticais ou de clareza
        3. Adaptar a linguagem para o público-alvo considerando idade e região
        4. Adicionar termos técnicos relevantes quando apropriado
        5. Aprofundar o conteúdo quando necessário

        Para cada modificação realizada, você deve classificá-la em uma das três categorias:
        - Adequação para melhor compreensão
        - Correção de erro gramatical
        - Adição para maior profundidade

        Forneça sua resposta no formato JSON com as seguintes estruturas:
        {
          "texto_modificado": "Texto completo com todas as alterações",
          "modificacoes": [
            {
              "original": "texto original",
              "modificado": "texto modificado",
              "tipo": "compreensao|gramatica|profundidade",
              "justificativa": "explicação da modificação"
            }
          ]
        }
        """,
        description="Agente que analisa e adapta textos para um objetivo específico"
    )

    entrada = f"""
    Texto Original:
    {texto_original}

    Informações do Usuário:
    Idade: {idade}
    Estado: {estado}
    Área de Atuação: {area}
    Objetivo: {objetivo}

    Informações sobre Linguagem Regional:
    {info_linguagem}

    Informações sobre a Área e Objetivo:
    {info_area}

    Informações sobre Termos Comuns:
    {info_termos}
    """

    resultado = call_agent(analisador, entrada)
    return resultado

# --- AGENTE 5: GERADOR DE HTML, CSS E JS ---
def agente_gerador_html(resultado_analise):
    gerador_html = Agent(
        name="gerador_html",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em geração de código HTML, CSS e JavaScript. Sua tarefa é criar
        um interface web interativa que destaque as modificações feitas em um texto.

        Você receberá um JSON contendo um texto modificado e uma lista de modificações.
        Para cada modificação, você deve:

        1. Envolver o texto modificado em uma tag <span> com classe apropriada:
           - "azul-claro" para modificações do tipo "compreensao"
           - "laranja-claro" para modificações do tipo "gramatica"
           - "verde-claro" para modificações do tipo "profundidade"
        2. Adicionar o atributo "title" à tag span contendo o texto original, para exibir quando
           o usuário passar o mouse por cima
        3. Implementar interatividade adicional usando JavaScript para melhorar a experiência
           do usuário

        Forneça sua resposta como um código HTML completo, incluindo CSS e JavaScript em um único arquivo.
        O resultado deve ser visualmente atraente e funcional.
        """,
        description="Agente que gera código HTML, CSS e JavaScript para visualização de textos modificados"
    )

    entrada = f"""
    Resultado da análise de texto (em formato JSON):
    {resultado_analise}

    Crie um HTML completo com CSS e JavaScript que:
    1. Mostre o texto modificado com destaques coloridos para cada tipo de modificação
    2. Permita ver o texto original ao passar o mouse sobre o texto modificado
    3. Adicione uma legenda explicando o significado das cores
    4. Inclua recursos interativos para melhorar a experiência do usuário
    5. Ao final sugira dicas rápidas e simpes para ter uma boa comunicação e não ficar nervoso, técnicas de respiração e outras que se adequar para a situação.
    """

    try:
        # Chama o agente para gerar o HTML
        resultado = call_agent(gerador_html, entrada)
        return resultado
    except Exception as e:
        # Em caso de erro na geração do HTML
        # Tenta converter o resultado para JSON
        try:
            analise = json.loads(resultado_analise)

            texto_modificado = analise.get("texto_modificado", "")
            modificacoes = analise.get("modificacoes", [])

            # Aplica cada modificação com as classes corretas
            for mod in modificacoes:
                original = mod.get("original", "")
                modificado = mod.get("modificado", "")
                tipo = mod.get("tipo", "")
                justificativa = mod.get("justificativa", "")

                # Define a classe CSS baseada no tipo
                classe = ""
                if tipo == "compreensao":
                    classe = "azul-claro"
                elif tipo == "gramatica":
                    classe = "laranja-claro"
                elif tipo == "profundidade":
                    classe = "verde-claro"

                # Cria a tag span com a classe apropriada
                if classe and original in texto_modificado:
                    texto_modificado = texto_modificado.replace(
                        modificado,
                        f'<span class="{classe}" title="{original}">{modificado}</span>'
                    )

            # Adiciona o CSS para o resultado final
            html_final = f"""
            <style>
            .azul-claro {{
                background-color: #add8e6;
                padding: 2px;
                border-radius: 3px;
                cursor: help;
            }}
            .laranja-claro {{
                background-color: #ffd580;
                padding: 2px;
                border-radius: 3px;
                cursor: help;
            }}
            .verde-claro {{
                background-color: #90ee90;
                padding: 2px;
                border-radius: 3px;
                cursor: help;
            }}
            span[title] {{
                position: relative;
            }}
            span[title]:hover::after {{
                content: attr(title);
                position: absolute;
                bottom: 100%;
                left: 0;
                background-color: #333;
                color: white;
                padding: 5px;
                border-radius: 3px;
                white-space: nowrap;
                z-index: 10;
            }}
            </style>
            <div class="resultado-analise">
                <h3>Texto Adaptado e Otimizado</h3>
                <p>{texto_modificado}</p>

                <h3>Legenda</h3>
                <p><span class="azul-claro">Texto em azul</span>: Modificado para melhor compreensão</p>
                <p><span class="laranja-claro">Texto em laranja</span>: Correção gramatical</p>
                <p><span class="verde-claro">Texto em verde</span>: Adicionado para maior profundidade</p>
            </div>
            """
            return html_final
        except Exception as e:
            # Em caso de erro no processamento do JSON
            return f"<p>Erro ao gerar HTML: {str(e)}</p><p>Resultado bruto da análise:</p><pre>{resultado_analise}</pre>"

print("Data atual:", data_de_hoje)
print("\n--- Por favor, preencha as informações abaixo ---")
# Coleta de informações do usuário
idade = input("❓ Idade aproximada do público-alvo: ")
estado = input("❓ Estado onde vive: ")
area = input("❓ Área de atuação/interesse: ")
objetivo = input("❓ Objetivo do texto (ex: daily, apresentação, ensinar): ")

# Texto a ser analisado
print("\n--- Digite ou cole o texto que você deseja analisar e adaptar ---")
print("(Pressione Enter duas vezes para adicionar o texto)")

texto_linhas = []
while True:
    linha = input()
    if linha == "":
        break
    texto_linhas.append(linha)

texto_original = "\n".join(texto_linhas)

if not texto_original:
    print("Você não inseriu nenhum texto para análise!")
else:
    print("\n--- Processando sua solicitação... ---")

    # Executa os agentes de pesquisa
    print("\n--- 🔍 Pesquisando padrões de linguagem... ---")
    info_linguagem = agente_pesquisador_linguagem(idade=idade, estado=estado)
    display(to_markdown(info_linguagem))

    print("\n--- 🔍 Pesquisando informações sobre área e objetivo... ---")
    info_area = agente_pesquisador_area_obje(area=area, objetivo=objetivo)  # Corrigido: adicionada a chamada de função
    display(to_markdown(info_area))

    print("\n--- 🔍 Pesquisando termos comuns utilizados... ---")
    info_termos = agente_pesquisador_termos_comuns(idade, estado, area)
    display(to_markdown(info_termos))

    # Executa o agente de análise de texto
    print("\n--- 📝 Analisando e adaptando o texto... ---")
    resultado_analise = agente_analisador_texto(
        texto_original, idade, estado, area, objetivo,
        info_linguagem, info_area, info_termos
    )

    # Executa o agente gerador de HTML
    print("\n--- 🌐 Gerando visualização HTML interativa... ---")
    html_final = agente_gerador_html(resultado_analise)

    # Gera e exibe o HTML final
    print("\n--- ✨ Resultado Final ✨ ---")
    display(HTML(html_final))

Data atual: 17/05/2025

--- Por favor, preencha as informações abaixo ---
❓ Idade aproximada do público-alvo: 55
❓ Estado onde vive o público-alvo: recife
❓ Área de atuação/interesse: dev
❓ Objetivo do texto (ex: convencer, informar, entreter): daily

--- Digite ou cole o texto que você deseja analisar e adaptar ---
(Pressione Enter duas vezes para finalizar)
Oi gente, então, tamo aqui hoje pra falar de umas coisas aí da economia da empresa, que tá meio enrolada ou talvez não, depende do ponto de vista, né. A gente vai tentar entender uns negócios de números, lucros e outras paradas tipo gastos e tal, que são importantes aí pra, tipo, ver como que tá indo as coisas, sabe?  Então, vamo começar logo que já tá todo mundo meio cansado e é melhor resolver isso rapidão. Quem tiver alguma coisa pra falar, fala aí, e se não tiver, só escuta mesmo. Bora lá.


--- Processando sua solicitação... ---

--- 🔍 Pesquisando padrões de linguagem... ---


> Ok, vamos elaborar um roteiro de pesquisa para encontrar os termos, gírias e padrões de linguagem mais recentes utilizados por pessoas com aproximadamente 55 anos no estado de Recife.
> 
> 
> Com base na sua solicitação, organizei um roteiro de pesquisa sobre termos, gírias e padrões de linguagem utilizados por pessoas com aproximadamente 55 anos no estado de Recife.
> 
> **1. Gírias e expressões populares em Recife:**
> 
> *   **Objetivo:** Identificar as gírias e expressões mais comuns usadas pela população de Recife, com foco em pessoas de 55 anos.
> *   **Termos de pesquisa:**
> 
>     *   "gírias e expressões populares Recife"
>     *   "linguagem informal Recife"
>     *   "expressões típicas Recife"
>     *   "o que significa \[gíria específica de Recife]"
> *   **Fontes:** Blogs locais, dicionários de gírias regionais, artigos sobre a cultura de Recife.
> 
> **2. Padrões de comunicação e linguagem da faixa etária:**
> 
> *   **Objetivo:** Analisar se existem padrões de comunicação específicos para pessoas de 55 anos em Recife.
> *   **Termos de pesquisa:**
> 
>     *   "padrões de comunicação Recife"
>     *   "linguagem e gerações Recife"
>     *   "vocabulário de pessoas de 50+ em Recife"
> *   **Fontes:** Estudos sociolinguísticos, artigos acadêmicos sobre linguagem e idade, entrevistas com moradores de Recife.
> 
> **3. Contextos de uso da linguagem:**
> 
> *   **Objetivo:** Entender como as gírias e expressões são empregadas em situações informais e formais.
> *   **Termos de pesquisa:**
> 
>     *   "uso de gírias em Recife"
>     *   "exemplos de conversas informais Recife"
>     *   "linguagem formal e informal em Recife"
> *   **Fontes:** Análise de redes sociais, transcrições de entrevistas, peças de teatro ou programas de TV locais.
> 
> **4. Termos, gírias e expressões relevantes:**
> 
> *   **Objetivo:** Criar uma lista organizada dos termos, gírias e expressões mais relevantes, incluindo seus significados e contextos de uso.
> *   **Termos de pesquisa:**
> 
>     *   "dicionário de recifês"
>     *   "gírias mais usadas em Recife"
>     *   "expressões antigas de Recife"
> *   **Fontes:** Dicionários de gírias, artigos sobre a cultura local, entrevistas com moradores de Recife.
> 
> **Exemplos de gírias e expressões de Recife (com base nos resultados da pesquisa):**
> 
> *   **Massa:** Algo bom, da hora, interessante. Exemplo: "A festa ontem à noite foi massa demais".
> *   **Tá ligado?:** Pergunta retórica para confirmar se a pessoa entendeu o que foi dito.
> *   **Mano véi:** Amigo antigo.
> *   **Arretado:** Muito bom, excelente, maravilhoso. Também pode significar irritado.
> *   **Cabra da peste:** Pessoa habilidosa, esperta.
> *   **Oxe/Oxente:** Interjeições de espanto.
> *   **Visse?:** Viu?
> *   **Abestado/Tabacudo:** Bobo, tolo, idiota.
> *   **Pirangueiro:** Pessoa pão-duro, mão de vaca, avarento.
> *   **Xexero:** Pessoa devedora, que não gosta de pagar dívidas.
> *   **Buliçoso:** Aquele que gosta de mexer em tudo.
> *    **Farda:** Uniforme escolar.
> *   **Pitó:** Elástico de cabelo.
> 
> **Observações:**
> 
> *   A linguagem está em constante evolução, por isso, é importante buscar fontes recentes para obter informações atualizadas.
> *   As gírias podem ter diferentes significados dependendo do contexto e da intenção do falante.
> *   É fundamental respeitar as diferenças regionais e culturais ao utilizar gírias e expressões locais.
> 
> Este roteiro de pesquisa fornece um ponto de partida para entender os termos, gírias e padrões de linguagem utilizados por pessoas com aproximadamente 55 anos em Recife. Ao explorar as fontes mencionadas e realizar entrevistas com moradores locais, será possível obter uma compreensão mais completa e precisa da linguagem da região.
> 



--- 🔍 Pesquisando informações sobre área e objetivo... ---


> Para encontrar as informações mais recentes e relevantes sobre "daily" na área de "dev", vou usar o Google para pesquisar os seguintes tópicos:
> 
> 1.  O que são dailies em desenvolvimento de software e por que são importantes?
> 2.  Quais são as melhores práticas para conduzir uma daily eficaz?
> 3.  Como as ferramentas de gerenciamento de projetos podem auxiliar nas dailies?
> 4.  Quais são os erros comuns em dailies e como evitá-los?
> 
> 
> Com base nas pesquisas, aqui estão algumas das informações mais recentes e relevantes sobre dailies na área de desenvolvimento:
> 
> **1. O que é uma Daily e por que é importante?**
> 
> *   A Daily Scrum (ou Daily Meeting) é uma reunião diária e rápida (geralmente 15 minutos) que visa alinhar a equipe, promover a transparência, comunicação e colaboração. Ela permite que todos estejam na mesma página e ajuda a identificar e resolver problemas rapidamente.
> 
> *   As Dailies são um pilar da cultura ágil, promovendo a inspeção e adaptação contínua.
> 
> **2. Melhores práticas para conduzir uma Daily eficaz:**
> 
> *   **Foco na meta da Sprint:** Avaliar o progresso para atingir a meta da Sprint, revisando e planejando atividades para as próximas 24 horas.
> 
> *   **Identificar impedimentos:** A Daily não é uma reunião de status report, mas sim um momento para identificar impedimentos que dificultam o avanço.
> 
> *   **Ser conciso e objetivo:** Respeitar o timebox de 15 minutos, focando no que realmente importa e relatando avanços ou impedimentos.
> 
> *   **Criar um ambiente positivo:** Estimular a participação de todos, valorizar opiniões e criar um ambiente seguro para compartilhar dificuldades.
> 
> *   **Adaptar ao time:** A Daily deve ser adaptada às necessidades e características da equipe, não há uma fórmula única.
> 
> *   **Manter a regularidade:** Realizar a Daily sempre no mesmo local e horário para criar uma rotina para o time.
> 
> **3. Erros comuns em Dailies e como evitá-los:**
> 
> *   **Reunião demorada:** Manter o foco e o timebox de 15 minutos.
> 
> *   **Reunião de status report:** Concentrar-se no progresso em direção à meta da Sprint e nos impedimentos.
> 
> *   **Reunião para resolver problemas:** Levantar os problemas e resolvê-los após a reunião com os envolvidos.
> 
> *   **Reunião para socialização:** Manter o foco nos objetivos da Daily.
> 
> *   **Falar demais:** Ser conciso e evitar monopolizar a reunião.
> 
> *   **Ser negativo:** Criar um ambiente motivador e focado na solução de problemas.
> 
> *   **Não seguir as regras:** Respeitar a estrutura da Daily e o tempo estipulado.
> 
> **4. Ferramentas de gerenciamento de projetos que auxiliam nas Dailies:**
> 
> *   Ferramentas como Trello, Asana, Jira, Click Up, Monday.com e Zoho Projects podem auxiliar no gerenciamento de tarefas, organização do fluxo de trabalho e comunicação da equipe.
> 
> *   Elas ajudam a manter a transparência, o controle de prazos e a organização das entregas.
> 
> **5. Dicas adicionais:**
> 
> *   **Modelo EVA (Entender, Visualizar e Adaptar):** Uma abordagem inovadora para transformar a Daily Scrum em um rito ágil que realmente melhora a produtividade do time.
> 
> *   **Incentivar a participação:** Dar a cada membro da equipe a oportunidade de compartilhar suas realizações e desafios, fazendo com que todos se sintam valorizados e envolvidos.
> 
> Espero que estas informações sejam úteis para otimizar suas dailies na área de desenvolvimento!
> 



--- 🔍 Pesquisando termos comuns utilizados... ---


> Com base nas buscas realizadas, listo alguns termos e informações que podem ser relevantes para desenvolvedores com cerca de 55 anos em Recife:
> 
> **Linguagens de programação e tecnologias:**
> 
> *   **Java e Angular:** Há vagas para desenvolvedores Java e Angular em Recife, indicando que essas tecnologias ainda são relevantes no mercado local.
> *   **Python:** Também há demanda por desenvolvedores Python, tanto para back-end quanto para automação e APIs.
> *   **Linux:** O domínio de Linux é visto como um diferencial para profissionais de TI.
> *   **SQL:** A construção de consultas a bancos de dados em SQL é uma habilidade importante.
> *   **Ferramentas de IA:** O uso de ferramentas de Inteligência Artificial no desenvolvimento está em alta.
> 
> **Termos e gírias locais (Recife):**
> 
> *   **Caba/Cabra:** Homem.
> *   **Cabra de peia:** Homem corajoso.
> *   **Cabueta:** Dedo-duro.
> *   **Abestalhado:** Bobo.
> *   **Fuleiro:** Pessoa que não cumpre o que promete ou algo de má qualidade.
> *   **Leso:** Bobo.
> *   **Oxe:** Expressão de surpresa ou espanto.
> *   **Pala:** Mentira, conversa mole.
> *   **Deu a mulesta/deu a porra:** Deu ruim, aconteceu algo grave.
> 
> **Outros pontos relevantes:**
> 
> *   **Mercado de trabalho:** Há oportunidades para diferentes níveis de experiência, desde júnior até arquiteto de software.
> *   **Transformação digital:** A Prefeitura do Recife tem investido em transformação digital e inovação no setor público.
> *   **Comércio local:** Pessoas com 55 anos ou mais tendem a ser mais fiéis ao comércio local.
> 
> É importante ressaltar que essa é apenas uma amostra dos termos e informações que podem ser relevantes. A melhor forma de se manter atualizado é acompanhar as novidades do mercado de tecnologia, participar de eventos e comunidades locais e estar atento às tendências da área.



--- 📝 Analisando e adaptando o texto... ---

--- 🌐 Gerando visualização HTML interativa... ---

--- ✨ Resultado Final ✨ ---
